# CopasiProcess Composer

In [1]:
import sys 

sys.path.insert(0, '..')

In [2]:
import os 
from basico import *
from process_bigraph import Composite, pf 
from biosimulator_processes.processes.copasi_process import CopasiProcess
from biosimulator_processes.process_types import MODEL_TYPE, CHANGES_SCHEMA

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


In [3]:
from biosimulator_processes.utils import generate_sed_model_config_schema as sed_schema, fetch_biomodel


model = fetch_biomodel('BIOMD0000000051')

get_species(model=model)

,compartment,type,unit,initial_concentration,initial_particle_number,initial_expression,expression,concentration,particle_number,rate,particle_number_rate,key,sbml_id
name,,,,,,,,,,,,,
Phosphoenol pyruvate,cytosol,reactions,mmol/l,2.670,1.607912e+21,,,2.670,1.607912e+21,-6.037777e+00,-3.636034e+21,Metabolite_0,cpep
Extracellular Glucose,extracellular,reactions,mmol/l,2.000,1.204428e+21,,,2.000,1.204428e+21,-9.294452e-02,-5.597250e+19,Metabolite_1,cglcex
Glucose-6-Phosphate,cytosol,reactions,mmol/l,3.480,2.095705e+21,,,3.480,2.095705e+21,6.037499e+00,3.635867e+21,Metabolite_2,cg6p
Pyruvate,cytosol,reactions,mmol/l,2.670,1.607912e+21,,,2.670,1.607912e+21,6.037777e+00,3.636034e+21,Metabolite_3,cpyr
Fructose-6-Phosphate,cytosol,reactions,mmol/l,0.600,3.613284e+20,,,0.600,3.613284e+20,-1.153453e-03,-6.946258e+17,Metabolite_4,cf6p
Glucose-1-Phosphate,cytosol,reactions,mmol/l,0.653,3.932458e+20,,,0.653,3.932458e+20,-4.253893e-05,-2.561754e+16,Metabolite_5,cg1p
6-Phosphogluconate,cytosol,reactions,mmol/l,0.808,4.865890e+20,,,0.808,4.865890e+20,-2.316249e-04,-1.394878e+17,Metabolite_6,cpg
"Fructose-1,6-bisphosphate",cytosol,reactions,mmol/l,0.272,1.638022e+20,,,0.272,1.638022e+20,1.133577e-03,6.826562e+17,Metabolite_7,cfdp
sedoheptulose-7-phosphate,cytosol,reactions,mmol/l,0.276,1.662111e+20,,,0.276,1.662111e+20,-1.956809e-11,-1.178418e+10,Metabolite_8,csed7p


In [4]:
get_parameters(model=model)

,type,unit,initial_value,initial_expression,expression,value,rate,key,sbml_id
name,,,,,,,,,
catp,assignment,mmol/l,4.270000,,4.2699999999999996 - 4.1630000000000003 * ( Ti...,4.270000,NaN,ModelValue_0,catp
cadp,assignment,mmol/l,0.582000,,0.58199999999999996 + 1.73 * 2.730999999999999...,0.582000,NaN,ModelValue_1,cadp
camp,assignment,mmol/l,0.954783,,0.123 + 7.25 * ( Time / ( 7.25 + 1.47 * Time +...,0.954783,NaN,ModelValue_2,camp
cnadp,assignment,mmol/l,0.196759,,0.159 - 0.0055399999999999998 * ( Time / ( 2.7...,0.196759,NaN,ModelValue_3,cnadp
cnadph,assignment,mmol/l,0.062000,,0.062 + 0.33200000000000002 * 2.718 ^ ( - 0.46...,0.062000,NaN,ModelValue_4,cnadph
cnad,assignment,mmol/l,1.464399,,1.3140000000000001 + 1.3140000000000001 * 2.73...,1.464399,NaN,ModelValue_5,cnad
cnadh,assignment,mmol/l,0.093400,,0.093399999999999997 + 0.0011100000000000001 *...,0.093400,NaN,ModelValue_6,cnadh


In [5]:
get_reactions(model=model)

,scheme,flux,particle_flux,function,key,sbml_id,mapping
name,,,,,,,
Phosphotransferase system,"""Extracellular Glucose"" + 65 * ""Phosphoenol py...",9.597360e-02,5.779665e+19,Function for Phosphotransferase system,Reaction_0,vPTS,"{'KPTSa1': 3082.3, 'KPTSa2': 0.01, 'KPTSa3': 2..."
Glucose-6-phosphate isomerase,Glucose-6-Phosphate = Fructose-6-Phosphate; 6...,5.837679e-02,3.515532e+19,Function for Glucose-6-phosphate isomerase,Reaction_1,vPGI,"{'KPGIeq': 0.1725, 'KPGIf6p': 0.266, 'KPGIf6pp..."
Phosphoglucomutase,Glucose-6-Phosphate = Glucose-1-Phosphate,2.292288e-03,1.380448e+18,Function for Phosphoglucomutase,Reaction_2,vPGM,"{'KPGMeq': 0.196, 'KPGMg1p': 0.0136, 'KPGMg6p'..."
Glucose-6-phosphate dehydrogenase,Glucose-6-Phosphate = 6-Phosphogluconate,1.400197e-01,8.432183e+19,Function for Glucose-6-phosphate dehydrogenase,Reaction_3,vG6PDH,"{'KG6PDHg6p': 14.4, 'KG6PDHnadp': 0.0246, 'KG6..."
Phosphofructokinase,"Fructose-6-Phosphate = Fructose-1,6-bisphospha...",1.423843e-01,8.574585e+19,Function for Phosphofructokinase,Reaction_4,vPFK,"{'KPFKadpa': 128.0, 'KPFKadpb': 3.89, 'KPFKadp..."
Transaldolase,Glyceraldehyde-3-Phosphate + sedoheptulose-7-p...,4.531300e-02,2.728813e+19,Function for Transaldolase,Reaction_5,vTA,"{'KTAeq': 1.05, 'ce4p': 'Erythrose-4-phosphate..."
Transketolase a,Ribose-5-phosphate + Xylulose-5-phosphate = Gl...,4.532067e-02,2.729275e+19,Function for Transketolase a,Reaction_6,vTKA,"{'KTKaeq': 1.2, 'cgap': 'Glyceraldehyde-3-Phos..."
Transketolase b,Erythrose-4-phosphate + Xylulose-5-phosphate =...,3.843200e-02,2.314429e+19,Function for Transketolase b,Reaction_7,vTKB,"{'KTKbeq': 10.0, 'ce4p': 'Erythrose-4-phosphat..."
Mureine synthesis,2 * Fructose-6-Phosphate =,4.371100e-04,2.632338e+17,Constant flux (reversible),Reaction_8,vMURSyNTH,{'v': 0.00043711}


{'model': {'model_changes': {'species_changes': {'Extracellular Glucose': {'initial_concentration': 5.0}},
   'parameter_changes': {'catp': {'initial_value': 100.0}},
   'reaction_changes': {'Aldolase': {'scheme': 'A -> B'}}}},
 'biomodel_id': 'BIOMD0000000051'}

### Example 1: Define a single process instance and a corresponding emitter

##### 1. Define the schema by which our Composite instance will be configured. Here a user chooses an entrypoint through which to instantiate the composite process. We will use an SBML-encoded model file as our entrypoint in this example.


##### **As per the `CopasiProcess` constructor, there are three primary methods of entry:**
    

    # Parse values from the process `config` attribute dictionary:
    
        model_file = self.config.get('model').get('model_source')
        sed_model_id = self.config.get('model').get('model_id')
        biomodel_id = self.config.get('biomodel_id')
        source_model_id = biomodel_id or sed_model_id


    # A. enter with model_file
    
        if model_file:
            self.copasi_model_object = load_model(model_file)
    

    # B. enter with specific search term for a model
     
        elif source_model_id:
            self.copasi_model_object = fetch_biomodel(model_id=source_model_id)
    

    # C. enter with a new model
    
        else:
            self.copasi_model_object = new_model(name='CopasiProcess Model')

##### Here we define the schema for a single process based on the provided entrypoint.

In [ ]:
root = os.path.abspath('..')
model_path = os.path.join(root, 'biosimulator_processes/model_files/BIOMD0000000061_url.xml')

instance1 = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_source': model_path
            }
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}

##### 2. Instantiate the Process-Bigraph-Engine with the Composite class

In [ ]:
workflow = Composite({
    'state': instance1
})

##### 3. Run the Composite workflow with the Engine

In [ ]:
workflow.run(10)

##### 4. Gather and neatly-print the results.

In [ ]:
results = workflow.gather_results()

print(f'RESULTS: {pf(results)}')

##### 5. Plot/Analyze the results with Plotly

In [ ]:
# 5. Plot/Analyze the results

#### Example 2: Define a single-process composition from a given BioModels model id and make model changes. The options available for configuration of a `'model'` that can be run by the composite process are based on SEDML L1V4 specifications. Please note that not all parameters are required in this configuration.

In [ ]:
CHANGES_SCHEMA

In [ ]:
instance2 = {
    'copasi': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_changes': {
                    'reaction_changes': {
                        'R1': 'A -> B'
                    }
                }
            }
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}


results = play_composition(instance2, 10)

#### Example 2a: Define a single process with a different entrypoint.

#### Example 3: Define multiple processes in the same instance for a parameter scan. 

In [3]:
from biosimulator_processes.utils import generate_copasi_process_instance

In [1]:
PARAM SCAN FROM LOADED BIOMODEL HERE:




SyntaxError: invalid syntax (2637470785.py, line 1)

In [1]:
parameter_scan_instance = {
    'copasi_1': {
        '_type': 'process',
        'address': 'local:copasi',
        'config': {
            'model': {
                'model_changes': {
                    'reaction_changes': {
                        'R1': 'A -> B'
                    }
                }
            }
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'model_parameters': ['model_parameters_store'],
            'time': ['time_store'],
            'reactions': ['reactions_store']
        },
        'outputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'floating_species': 'tree[float]',
                'time': 'float',
            },
        },
        'inputs': {
            'floating_species': ['floating_species_store'],
            'time': ['time_store'],
        }
    }
}

In [1]:
from biosimulator_processes.utils import generate_composite_copasi_process_instance as copasi_instance

In [2]:
model_config = {
    'model': {
        'model_changes': {
            'reaction_changes': {
                'R1': 'A -> B'
            }
        }
    }
}


instance = copasi_instance(
    instance_name='copasi_A',
    config=model_config,
    add_emitter=True
)

In [3]:
instance

{'copasi_A': {'_type': 'process',
  'address': 'local:copasi',
  'config': {'model': {'model_changes': {'reaction_changes': {'R1': 'A -> B'}}}},
  'inputs': {'floating_species': ['floating_species_store'],
   'model_parameters': ['model_parameters_store'],
   'time': ['time_store'],
   'reactions': ['reactions_store']},
  'outputs': {'floating_species': ['floating_species_store'],
   'time': ['time_store']}},
 'emitter': {'_type': 'step',
  'address': 'local:ram-emitter',
  'config': {'emit': {'floating_species': 'tree[float]', 'time': 'float'}},
  'inputs': {'floating_species': ['floating_species_store'],
   'time': ['time_store']}}}